In [ ]:
%load_ext autoreload
%autoreload 2

from mimic_iv_analysis.configurations.params import DEFAULT_STUDY_TABLES_LIST
from mimic_iv_analysis.io import DataLoader, TableNames, ParquetConverter, ExampleDataLoader
import pandas as pd

import dask.dataframe as dd

# from dask.distributed import Client
# client = Client(n_workers=4, threads_per_worker=4, processes=True, memory_limit='8GB')
# client

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
example = ExampleDataLoader(partial_loading=False, num_subjects=10, apply_filtering=True)

In [17]:
df = example.load_merged_tables()

[21:20:45] INFO     Loaded 2423918 rows for 10 subjects from merged table.                       data_loader.py:525

[21:20:51] INFO     Loaded 2423918 rows after partial loading.                                   data_loader.py:529

In [19]:
df.shape[0].compute()

KeyboardInterrupt: 

## Scan the directory

In [2]:
from pathlib import Path

MIMIC_DATA_PATH = Path("/Users/artinmajdi/Documents/GitHubs/RAP/mimic__pankaj/dataset/mimic-iv-3.1")

# Initialize DataLoader and scan directory
loader = DataLoader(mimic_path=MIMIC_DATA_PATH)
loader.scan_mimic_directory()



In [3]:
# # Create ParquetConverter
# converter = ParquetConverter(data_loader=loader)

# # Example: Save a small table as Parquet
# converter.save_as_parquet(table_name=TableNamesHOSP.ADMISSIONS)

In [ ]:
# TODO: Load tables individually and merge them with the how=outer (filters applied after merging) and see if it makes any difference compare to when I do it with the inner (filters applied before merging).

examples_full = ExampleDataLoader(partial_loading=False)
# examples_partial = ExampleDataLoader(partial_loading=True, num_subjects=10, random_selection=False)

In [ ]:
examples_partial.study_table_info()

In [ ]:
examples_partial.counter()

In [ ]:
df = examples_partial.load_merged_tables(partial_loading=True, num_subjects=10, random_selection=False, use_dask=True)

In [ ]:
# Print in a tabular format
examples_partial.n_rows_after_merge()